# Text summarization using extractive method 
- Skipthoughts, tfidf-w2v embading

## Test with email/chat example example

## Skipthoughts model

In [1]:
import skipthoughts
model = skipthoughts.load_model()
encoder = skipthoughts.Encoder(model)

Loading model parameters...
Compiling encoders...
Loading tables...
Packing up...


In [2]:
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min


In [3]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import re
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import gensim
from gensim import corpora
from gensim.models import Phrases
from gensim.models import CoherenceModel
from nltk.stem import PorterStemmer


import numpy as np
import nltk
import scipy as sp
import pandas as pd
import matplotlib
#import pytextrank
#import pytldr
import sys
import spacy
import theano
import tornado.ioloop
import tornado.web
import re
import os
from __future__ import division
from nltk.tokenize import word_tokenize
import theano.tensor as T
from tornado.options import define, options

## read the text

In [4]:
# one email as text and convert it as skipthought vectors
# each sentence will be repreented by skipthought embading vectors in 4800 dimentions
# skipthought use pretrain model
text="Thank you for keeping me updated on this issue. I'm happy to hear that the issue got resolved after all and you can now use the app in its full functionality again. Also many thanks for your suggestions. We hope to improve this feature in the future. In case you experience any further problems with the app, please don't hesitate to contact me again."

text="hello i would like to add and activate an old device on my account, but I can't find the XXXXXXXXX and since the iphone is disabled, I can't get in the phone. how can I add this phone.  I also don't have the box can the info be looked up. the phone is disabled and it keeps telling me totry again in 15 mins since this is an old device from this account. can we look up the info. yes mariviel xxxx i am pretty sure it is an Apple iPhone6 and it was used for line ending in NNNN. ok i would like to activate it on a new line. line NNNN just got the current phone, so it is the one. before it i am having trouble opening the phone on the back of the phone. i see Model ANNNN...does that help. ph gosh...i think i found it. is it on the back of the phone. is see a bunch of numbers amp. letters FCC ID BCG ENNNNA it's so tiny. hopefully i am seeing it correctly all of them. oh great!  what phone is this. yes! got it. i also see it on the phone. can I use the sim card in here. awesome says I need a SIMS ID NNNNNNNNNNNNNNN. is the XXXXXX on the card. do i enter all the number.  Says it is not vaild a new line can you help me with this. add a new line. oh ok no.  you have been very helpful. thanks you too"

In [5]:
text = sent_tokenize(text, language = 'english')
vectors = encoder.encode(text)
print(len(vectors))
print(vectors.shape)

37
7
14
24
22
12
16
9
8
10
3
5
2
15
4
6
29
(29, 4800)


In [6]:
## find the number of clusters base on number of vectors

n_clusters = int(np.ceil(len(vectors)**0.5))
kmeans = KMeans(n_clusters=n_clusters)
kmeans = kmeans.fit(vectors)


avg = []
for j in range(n_clusters):
    idx = np.where(kmeans.labels_ == j)[0]
    avg.append(np.mean(idx))
#Compute minimum distances between one point and a set of points.
#This function computes for each row in X,
#the index of the row of Y which is closest
#(according to the specified distance). 
#The minimal distances arealso returned.
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, vectors)
ordering = sorted(range(n_clusters), key=lambda k: avg[k])


summary = ' '.join([text[closest[idx]] for idx in ordering])

In [7]:
summary

"hello i would like to add and activate an old device on my account, but I can't find the XXXXXXXXX and since the iphone is disabled, I can't get in the phone. yes mariviel xxxx i am pretty sure it is an Apple iPhone6 and it was used for line ending in NNNN. Says it is not vaild a new line can you help me with this. is it on the back of the phone. oh great! thanks you too"

## TFIDF and Word2Vec embading and summerization


In [9]:
#text="Thank you for keeping me updated on this issue. I'm happy to hear that the issue got resolved after all and you can now use the app in its full functionality again. Also many thanks for your suggestions. We hope to improve this feature in the future. In case you experience any further problems with the app, please don't hesitate to contact me again."
text="hello i would like to add and activate an old device on my account, but I can't find the XXXXXXXXX and since the iphone is disabled, I can't get in the phone. how can I add this phone.  I also don't have the box can the info be looked up. the phone is disabled and it keeps telling me totry again in 15 mins since this is an old device from this account. can we look up the info. yes mariviel xxxx i am pretty sure it is an Apple iPhone6 and it was used for line ending in NNNN. ok i would like to activate it on a new line. line NNNN just got the current phone, so it is the one. before it i am having trouble opening the phone on the back of the phone. i see Model ANNNN...does that help. ph gosh...i think i found it. is it on the back of the phone. is see a bunch of numbers amp. letters FCC ID BCG ENNNNA it's so tiny. hopefully i am seeing it correctly all of them. oh great!  what phone is this. yes! got it. i also see it on the phone. can I use the sim card in here. awesome says I need a SIMS ID NNNNNNNNNNNNNNN. is the XXXXXX on the card. do i enter all the number.  Says it is not vaild a new line can you help me with this. add a new line. oh ok no.  you have been very helpful. thanks you too"
text = sent_tokenize(text, language = 'english')
print(text)

text_lis=[]
for i in text:
    text_lis.append(i.split())

#text_lis

["hello i would like to add and activate an old device on my account, but I can't find the XXXXXXXXX and since the iphone is disabled, I can't get in the phone.", 'how can I add this phone.', "I also don't have the box can the info be looked up.", 'the phone is disabled and it keeps telling me totry again in 15 mins since this is an old device from this account.', 'can we look up the info.', 'yes mariviel xxxx i am pretty sure it is an Apple iPhone6 and it was used for line ending in NNNN.', 'ok i would like to activate it on a new line.', 'line NNNN just got the current phone, so it is the one.', 'before it i am having trouble opening the phone on the back of the phone.', 'i see Model ANNNN...does that help.', 'ph gosh...i think i found it.', 'is it on the back of the phone.', 'is see a bunch of numbers amp.', "letters FCC ID BCG ENNNNA it's so tiny.", 'hopefully i am seeing it correctly all of them.', 'oh great!', 'what phone is this.', 'yes!', 'got it.', 'i also see it on the phone.

## word2vec

In [10]:
# word2vec and fasttext
from gensim.models import Word2Vec
from gensim.models import FastText

model_w2v = Word2Vec(sentences=text_lis, size=2000, window=5, min_count=1, workers=4, sg=0)
#model_w2v = FastText(sentences=text_lis, size=2000, window=5, min_count=1, workers=4,sg=0)


In [11]:
# create dictionary of words index pairs
lst_w2v = model_w2v.wv.index2word
dic_w2v = {ii: indx for indx, ii in enumerate(lst_w2v)}
len(dic_w2v)

141

In [12]:
w2v_matrix=model_w2v.wv.syn0
w2v_matrix.shape

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


(141, 2000)

## tfidf

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,TfidfTransformer

tf = TfidfVectorizer(ngram_range=(1, 1),vocabulary=dic_w2v)
tf_matrix = tf.fit_transform(text)

In [14]:
# features name
lst_tfidf = tf.get_feature_names()
x_df = pd.DataFrame(tf_matrix.toarray(), columns=lst_tfidf)
x_df.columns

Index(['the', 'is', 'it', 'i', 'on', 'I', 'phone.', 'can', 'a', 'and',
       ...
       'not', 'vaild', 'help', 'with', 'no.', 'been', 'very', 'helpful.',
       'thanks', 'too'],
      dtype='object', length=141)

In [15]:
## check both words and index are same in tfidf and word2vec output
ff1 = [0 if lst_w2v[ii]==x_df.columns[ii] else 1 for ii in range(len(lst_w2v))]
ff1.count(0)

141

## tfidf+w2v

In [ ]:
# multipying tfidf and w2v to get embading vector

m_tf = tf_matrix.toarray()
n_w2v=w2v_matrix
tf_w2v=np.matmul(tf_matrix.toarray(),w2v_matrix)
tf_w2v

## cluster and text summary

In [17]:
## find the number of clusters base on number of vectors

n_clusters = int(np.ceil(len(tf_w2v)**0.5))
kmeans = KMeans(n_clusters=n_clusters)
kmeans = kmeans.fit(tf_w2v)


avg = []
for j in range(n_clusters):
    idx = np.where(kmeans.labels_ == j)[0]
    avg.append(np.mean(idx))
#Compute minimum distances between one point and a set of points.
#This function computes for each row in X,
#the index of the row of Y which is closest
#(according to the specified distance). 
#The minimal distances arealso returned.
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, tf_w2v)
ordering = sorted(range(n_clusters), key=lambda k: avg[k])


summary = ' '.join([text[closest[idx]] for idx in ordering])

In [18]:
summary

'what phone is this. can we look up the info. is it on the back of the phone. add a new line. you have been very helpful. oh great!'

## text summary results comparision

# Verizon customer chat 

In [4]:
df_chat=pd.read_csv('/Users/akshay/Desktop/Fractal/NLP/chat_table_2018_12_12to2018_12_13_v1.csv',)
df_chat.head()

,Unnamed: 0,engagement_id,scat_event_dt,nps_score,chat_agent_name,cust_cb_sentiment_score,agent_cb_sentiment_score,customer_chat_text,agent_chat_text
0,0,7.237730e+17,12/13/2018,NaN,Rafzam Joni Abiog,NaN,NaN,hello i would like to add/activate an old devi...,I?ll be happy to help you today! As a courtesy...
1,1,7.237730e+17,12/13/2018,NaN,Ronald Duhon Iii,NaN,NaN,Is the website down? Is your website down It k...,"Welcome to Verizon Wireless Sales Chat, I'll b..."
2,2,7.237730e+17,12/13/2018,NaN,Jhemeil Tesiorna,NaN,NaN,Why does a number in my call log come up as NN...,"Hi, my name is Jamie. I am more than happy to ..."
3,3,7.237730e+17,12/13/2018,NaN,Mary Sansom,NaN,NaN,what is the galaxy note 9'XXXX storage capasit...,I?d be happy to see how I can assist you! With...
4,4,7.237730e+17,12/13/2018,NaN,Marcella Cook,NaN,NaN,Hello Jessica. Apparently there is an issue wi...,"Welcome to Verizon Wireless Sales Chat, I'll b..."


## Text preprocessing 

###  add punctuations in the chat data using punct library
- explore any other method we can use it 

In [12]:
#os.chdir('/Users/akshay/Downloads/skip-thoughts-master')
os.chdir('/Users/akshay/Desktop/Fractal/NLP/skip-thoughts-master/punctuator_clean')

In [13]:
import punct

Initializing ...
Number of parameters is 17926152


In [16]:
text1=punct.punctuate(df_chat.customer_chat_text[10])
text1

"I need help activating my new phone. I can't access my text messages, I'm trying to complete activation "

In [17]:
# function to get punctuation in the text
def add_punctuation(x):
    text=punct.punctuate(x)
    return text

In [18]:
df_chat1 = df_chat.dropna(axis=0, subset=['customer_chat_text'])
df_chat1['customer_chat_text_punctu']=df_chat1['customer_chat_text'].apply(add_punctuation)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
# list of tokenized sentence withing chat 
def text_list(x):
    text=sent_tokenize(x,language='english')
    text_lis=[]
    for i in text:
        text_lis.append(i.split())
    return text_lis

In [20]:
df_chat1['customer_chat_text_sent_list']=df_chat1['customer_chat_text_punctu'].apply(text_list)



/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
# list of plain text/sentence withing chat

def text_list_plain(x):
    text=sent_tokenize(x,language='english')
    text_lis=[]
    for i in text:
        text_lis.append(i)
    return text_lis

In [22]:
df_chat1['customer_chat_text_sent_list_plain']=df_chat1['customer_chat_text_punctu'].apply(text_list_plain)



/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
df_chat1

,Unnamed: 0,engagement_id,scat_event_dt,nps_score,chat_agent_name,cust_cb_sentiment_score,agent_cb_sentiment_score,customer_chat_text,agent_chat_text,customer_chat_text_punctu,customer_chat_text_sent_list,customer_chat_text_sent_list_plain
0,0,7.237730e+17,12/13/2018,NaN,Rafzam Joni Abiog,NaN,NaN,hello i would like to add/activate an old devi...,I?ll be happy to help you today! As a courtesy...,"Hello, i would like to add/activate an old dev...","[[Hello,, i, would, like, to, add/activate, an...","[Hello, i would like to add/activate an old de..."
1,1,7.237730e+17,12/13/2018,NaN,Ronald Duhon Iii,NaN,NaN,Is the website down? Is your website down It k...,"Welcome to Verizon Wireless Sales Chat, I'll b...","Is the website down, Is your website down? It ...","[[Is, the, website, down,, Is, your, website, ...","[Is the website down, Is your website down?, I..."
2,2,7.237730e+17,12/13/2018,NaN,Jhemeil Tesiorna,NaN,NaN,Why does a number in my call log come up as NN...,"Hi, my name is Jamie. I am more than happy to ...",Why does a number in my call log come up as NN...,"[[Why, does, a, number, in, my, call, log, com...",[Why does a number in my call log come up as N...
3,3,7.237730e+17,12/13/2018,NaN,Mary Sansom,NaN,NaN,what is the galaxy note 9'XXXX storage capasit...,I?d be happy to see how I can assist you! With...,"What is the galaxy note? 9'Xxxx storage, capas...","[[What, is, the, galaxy, note?], [9'Xxxx, stor...","[What is the galaxy note?, 9'Xxxx storage, cap..."
4,4,7.237730e+17,12/13/2018,NaN,Marcella Cook,NaN,NaN,Hello Jessica. Apparently there is an issue wi...,"Welcome to Verizon Wireless Sales Chat, I'll b...","Hello Jessica, Apparently there is an issue wi...","[[Hello, Jessica,, Apparently, there, is, an, ...","[Hello Jessica, Apparently there is an issue w..."
5,5,7.237730e+17,12/13/2018,NaN,Makeya Stanton,NaN,NaN,trying to setup a account gennia yes it said m...,I'll be happy to assist you today! May I have ...,"Trying to setup a account gennia. Yes, it said...","[[Trying, to, setup, a, account, gennia.], [Ye...","[Trying to setup a account gennia., Yes, it sa..."
6,6,7.237730e+17,12/13/2018,NaN,Shelby Mcgarity,NaN,NaN,I would like to see if qualify for 0 down Juli...,I?ll be happy to help you today! As a courtesy...,"I would like to see, if qualify for 0 down Jul...","[[I, would, like, to, see,, if, qualify, for, ...","[I would like to see, if qualify for 0 down Ju..."
7,7,7.237730e+17,12/13/2018,NaN,Jessie Woods,NaN,NaN,Hey i ordered an apple watch that said it was ...,I?ll be happy to help you today! As a courtesy...,"Hey, i ordered an apple watch. That said it wa...","[[Hey,, i, ordered, an, apple, watch.], [That,...","[Hey, i ordered an apple watch., That said it ..."
9,9,7.237730e+17,12/13/2018,NaN,Ralph Sorilla,NaN,NaN,control usage of granddaughter's phone Thelma ...,"Hi, my name is Jesse. I am more than happy to ...","Control usage of granddaughter's phone Thelma,...","[[Control, usage, of, granddaughter's, phone, ...",[Control usage of granddaughter's phone Thelma...
10,10,7.237730e+17,12/13/2018,NaN,Jessie Woods,NaN,NaN,I need help activating my new phone I can't ac...,I?ll be happy to help you today! As a courtesy...,I need help activating my new phone. I can't a...,"[[I, need, help, activating, my, new, phone.],...","[I need help activating my new phone., I can't..."


## W2V

In [32]:
# craete the training corpora for w2v model training

tem=[]
def create_list(x):
    for i in x:
        tem.append(i)

# updates list by applying function iterative on all the rows of the columns
df_chat1['customer_chat_text_sent_list'].apply(create_list)
        
training_corpora=tem.copy()
training_corpora

[['Hello,',
  'i',
  'would',
  'like',
  'to',
  'add/activate',
  'an',
  'old',
  'device',
  'on',
  'my',
  'account,',
  'but',
  'I',
  "can't",
  'find',
  'the',
  'XXXXXXXXX',
  '#',
  'and',
  'since',
  'the',
  'iphone',
  'is',
  'disabled,',
  'I',
  "can't",
  'get',
  'in',
  'the',
  'phone.'],
 ['How', 'can', 'I', 'add', 'this', 'phone?'],
 ['I', 'also', "don't", 'have', 'the', 'box?'],
 ['Can', 'the', 'info', 'be', 'looked', 'up?'],
 ['The',
  'phone',
  'is',
  'disabled',
  'and',
  'it',
  'keeps',
  'telling',
  'me',
  'totry',
  'again',
  'in',
  '15',
  'mins,',
  'since',
  'this',
  'is',
  'an',
  'old',
  'device.'],
 ['From', 'this', 'account.'],
 ['Can', 'we', 'look', 'up', 'the', 'info?'],
 ['Yes,', 'mariviel', 'xxxx.'],
 ['I',
  'am',
  'pretty',
  'sure',
  'it',
  'is',
  'an',
  'Apple',
  'iPhone6',
  'and',
  'it',
  'was',
  'used',
  'for',
  'line',
  'ending',
  'in',
  'NNNN.'],
 ['Ok,',
  'i',
  'would',
  'like',
  'to',
  'activate',
  '

In [34]:
# create word embading for all chat's words
# word2vec and fasttext
from gensim.models import Word2Vec
from gensim.models import FastText

model_w2v = Word2Vec(sentences=training_corpora, size=1000, window=5, min_count=1, workers=4, sg=0)
#model_w2v = FastText(sentences=text_lis, size=2000, window=5, min_count=1, workers=4,sg=0)

In [44]:
# create dictionary of words index pairs
lst_w2v = model_w2v.wv.index2word
dic_w2v = {ii: indx for indx, ii in enumerate(lst_w2v)}
len(dic_w2v)

w2v_matrix=model_w2v.wv.syn0
w2v_matrix.shape

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


(19771, 1000)

# tfidf

### (A) - tfidf for all sentence 

In [68]:
# craete training corpora for tfidf all sentence embading at a time
# create list of all sentence in the all chat 

tem=[]
def create_list(x):
    for i in x:
        tem.append(i)

# updates list by applying function iterative on all the rows of the columns
df_chat1['customer_chat_text_sent_list_plain'].apply(create_list)
        
training_corpora_list=tem.copy()
training_corpora_list

["Hello, i would like to add/activate an old device on my account, but I can't find the XXXXXXXXX # and since the iphone is disabled, I can't get in the phone.",
 'How can I add this phone?',
 "I also don't have the box?",
 'Can the info be looked up?',
 'The phone is disabled and it keeps telling me totry again in 15 mins, since this is an old device.',
 'From this account.',
 'Can we look up the info?',
 'Yes, mariviel xxxx.',
 'I am pretty sure it is an Apple iPhone6 and it was used for line ending in NNNN.',
 'Ok, i would like to activate it on a new line line.',
 'Nnnn just got the current phone, so it is the one before it.',
 'I am having trouble opening the phone on the back of the phone.',
 'I see Model ANNNN ....',
 'Does that help ph gosh ...?',
 'I think i found it.',
 "Is it on the back of the phone is see a bunch of numbers, amp amp letters, FCC ID BCG, ENNNNA, it's so tiny .... Hopefully i am seeing it correctly, all of them.",
 'Oh great.',
 'What phone is this?',
 'Yes 

In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,TfidfTransformer

tf = TfidfVectorizer(ngram_range=(1, 1),vocabulary=dic_w2v)
tf_matrix = tf.fit_transform(training_corpora_list) # passing all sentence 

tf_matrix.shape

In [ ]:
'''
## check both words and index are same in tfidf and word2vec output
# features name
lst_tfidf = tf.get_feature_names()
x_df = pd.DataFrame(tf_matrix.toarray(), columns=lst_tfidf)
x_df.columns

## check both words and index are same in tfidf and word2vec output
ff1 = [0 if lst_w2v[ii]==x_df.columns[ii] else 1 for ii in range(len(lst_w2v))]
ff1.count(0)
'''

### (B) tfidf sentence by sentence withing chat

In [87]:
#tfidf-w2v embading

# function to create tfidf vectors for each sentence withing chat 
# function to combined tfidf-w2v and create final embadings

tf = TfidfVectorizer(ngram_range=(1, 1),vocabulary=dic_w2v)

def tfidf_w2v_vector(x):
    tf_idf_vec=tf.fit_transform(x)
    m_tf = tf_idf_vec.toarray()
    n_w2v=w2v_matrix # w2v matrix
    tf_w2v=np.matmul(m_tf,n_w2v)
    result=tf_w2v
    return result
    

In [90]:
df_chat1['customer_chat_tfidf_w2v_embading']=df_chat1['customer_chat_text_sent_list_plain'].apply(tfidf_w2v_vector)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [91]:
df_chat1.head()

,Unnamed: 0,engagement_id,scat_event_dt,nps_score,chat_agent_name,cust_cb_sentiment_score,agent_cb_sentiment_score,customer_chat_text,agent_chat_text,customer_chat_text_punctu,customer_chat_text_sent_list,customer_chat_text_sent_list_plain,customer_chat_tfidf_w2v_embading
0,0,7.237730e+17,12/13/2018,NaN,Rafzam Joni Abiog,NaN,NaN,hello i would like to add/activate an old devi...,I?ll be happy to help you today! As a courtesy...,"Hello, i would like to add/activate an old dev...","[[Hello,, i, would, like, to, add/activate, an...","[Hello, i would like to add/activate an old de...","[[-0.02732193046301602, -0.5446890776415043, 0..."
1,1,7.237730e+17,12/13/2018,NaN,Ronald Duhon Iii,NaN,NaN,Is the website down? Is your website down It k...,"Welcome to Verizon Wireless Sales Chat, I'll b...","Is the website down, Is your website down? It ...","[[Is, the, website, down,, Is, your, website, ...","[Is the website down, Is your website down?, I...","[[0.017549944334763184, -0.13365034800907208, ..."
2,2,7.237730e+17,12/13/2018,NaN,Jhemeil Tesiorna,NaN,NaN,Why does a number in my call log come up as NN...,"Hi, my name is Jamie. I am more than happy to ...",Why does a number in my call log come up as NN...,"[[Why, does, a, number, in, my, call, log, com...",[Why does a number in my call log come up as N...,"[[-0.036100411540191925, -0.23755420301880667,..."
3,3,7.237730e+17,12/13/2018,NaN,Mary Sansom,NaN,NaN,what is the galaxy note 9'XXXX storage capasit...,I?d be happy to see how I can assist you! With...,"What is the galaxy note? 9'Xxxx storage, capas...","[[What, is, the, galaxy, note?], [9'Xxxx, stor...","[What is the galaxy note?, 9'Xxxx storage, cap...","[[0.017678189515359346, -0.08061104119938584, ..."
4,4,7.237730e+17,12/13/2018,NaN,Marcella Cook,NaN,NaN,Hello Jessica. Apparently there is an issue wi...,"Welcome to Verizon Wireless Sales Chat, I'll b...","Hello Jessica, Apparently there is an issue wi...","[[Hello, Jessica,, Apparently, there, is, an, ...","[Hello Jessica, Apparently there is an issue w...","[[0.11715480498819109, -0.03212657400216529, 0..."


In [92]:
# save data frame for future use 
df_chat1.to_csv('tfidf_w2v_output.csv')


### clustering and summary of the chat

In [101]:
## clustering and summary

import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

# function to get closest distance and order of the sentence
def summary(raw):
    text_encoded=raw['customer_chat_tfidf_w2v_embading']
    Customer_clean=raw['customer_chat_text_sent_list_plain']
    n_clusters = int(np.ceil(len(text_encoded)**0.5))
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans = kmeans.fit(text_encoded)
    avg = []
    for j in range(n_clusters):
        idx = np.where(kmeans.labels_ == j)[0]
        avg.append(np.mean(idx))
    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, text_encoded)
    ordering = sorted(range(n_clusters), key=lambda k: avg[k])
    summar = ' '.join([Customer_clean[closest[idx]] for idx in ordering])
    return summar


In [103]:
df_chat1['customer_chat_summary']=df_chat1.apply(summary,axis=1)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [125]:
df_chat1['customer_chat_length']=df_chat1.customer_chat_text_sent_list_plain.apply(lambda x:len(x))


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [126]:
df_chat1.head()

,Unnamed: 0,engagement_id,scat_event_dt,nps_score,chat_agent_name,cust_cb_sentiment_score,agent_cb_sentiment_score,customer_chat_text,agent_chat_text,customer_chat_text_punctu,customer_chat_text_sent_list,customer_chat_text_sent_list_plain,customer_chat_tfidf_w2v_embading,customer_chat_summary,customer_chat_length
0,0,7.237730e+17,12/13/2018,NaN,Rafzam Joni Abiog,NaN,NaN,hello i would like to add/activate an old devi...,I?ll be happy to help you today! As a courtesy...,"Hello, i would like to add/activate an old dev...","[[Hello,, i, would, like, to, add/activate, an...","[Hello, i would like to add/activate an old de...","[[-0.02732193046301602, -0.5446890776415043, 0...","Hello, i would like to add/activate an old dev...",30
1,1,7.237730e+17,12/13/2018,NaN,Ronald Duhon Iii,NaN,NaN,Is the website down? Is your website down It k...,"Welcome to Verizon Wireless Sales Chat, I'll b...","Is the website down, Is your website down? It ...","[[Is, the, website, down,, Is, your, website, ...","[Is the website down, Is your website down?, I...","[[0.017549944334763184, -0.13365034800907208, ...",It keeps telling me there is an error. I was j...,5
2,2,7.237730e+17,12/13/2018,NaN,Jhemeil Tesiorna,NaN,NaN,Why does a number in my call log come up as NN...,"Hi, my name is Jamie. I am more than happy to ...",Why does a number in my call log come up as NN...,"[[Why, does, a, number, in, my, call, log, com...",[Why does a number in my call log come up as N...,"[[-0.036100411540191925, -0.23755420301880667,...",Why does a number in my call log come up as NN...,3
3,3,7.237730e+17,12/13/2018,NaN,Mary Sansom,NaN,NaN,what is the galaxy note 9'XXXX storage capasit...,I?d be happy to see how I can assist you! With...,"What is the galaxy note? 9'Xxxx storage, capas...","[[What, is, the, galaxy, note?], [9'Xxxx, stor...","[What is the galaxy note?, 9'Xxxx storage, cap...","[[0.017678189515359346, -0.08061104119938584, ...","What is the galaxy note? 9'Xxxx storage, capas...",2
4,4,7.237730e+17,12/13/2018,NaN,Marcella Cook,NaN,NaN,Hello Jessica. Apparently there is an issue wi...,"Welcome to Verizon Wireless Sales Chat, I'll b...","Hello Jessica, Apparently there is an issue wi...","[[Hello, Jessica,, Apparently, there, is, an, ...","[Hello Jessica, Apparently there is an issue w...","[[0.11715480498819109, -0.03212657400216529, 0...","Hello Jessica, Apparently there is an issue wi...",7


In [127]:
df_chat1.to_csv('tfidf_w2v_output_result.csv')


In [123]:
!pwd

/Users/akshay/Downloads/skip-thoughts-master/punctuator_clean


In [107]:
df_chat1['customer_chat_text_sent_list_plain'][1]

['Is the website down, Is your website down?',
 'It keeps telling me there is an error.',
 'I was just wanting to see about trading in my phone for a non smartphone.',
 'How long do you think it will be before it is back up?',
 'Ok, thank you.']

In [108]:
df_chat1['customer_chat_summary'][1]

'It keeps telling me there is an error. I was just wanting to see about trading in my phone for a non smartphone. Ok, thank you.'